<a href="https://colab.research.google.com/github/NunnaPooja/DataExtraction_Python-scripts/blob/main/DataExtraction_InstaAcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Start WebDriver
driver = webdriver.Chrome(options=chrome_options)

def extract_instagram_data(profile_url, num_posts=10):
    driver.get(profile_url)
    time.sleep(5)  # Allow page to load

    # Extract Follower Count
    try:
        followers = driver.find_element(By.XPATH, '//header//span[contains(@title, "")]').get_attribute("title")
        print(f"Follower Count: {followers}")
    except:
        followers = "N/A"

    # Scroll to load posts dynamically
    post_links = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    while len(post_links) < num_posts:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)  # Wait for content to load
        posts = driver.find_elements(By.TAG_NAME, "a")
        new_links = [
            post.get_attribute("href")
            for post in posts
            if post.get_attribute("href") and "/p/" in post.get_attribute("href")
        ]
        post_links.extend(new_links)
        post_links = list(set(post_links))  # Remove duplicates
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # Check if scrolling has reached the end
            break
        last_height = new_height

    # Limit to the required number of posts
    post_links = post_links[:num_posts]

    # Initialize data storage
    data = []

    for post_url in post_links:
        driver.get(post_url)
        time.sleep(5)  # Allow post page to load

        try:

          # Extract Date
            try:
                date_element = driver.find_element(By.TAG_NAME, "time")
                date = date_element.get_attribute("datetime") if date_element else "N/A"
            except:
                date = "N/A"

            # Extract Post ID
            post_id = post_url.split("/p/")[1].strip("/")
            print(f"Processing Post ID: {post_id}")

            # Extract Likes
            try:
                likes_element = driver.find_element(By.XPATH, '//section//span[contains(text(), " likes")]')
                likes = likes_element.text.split(" ")[0] if likes_element else "0"
            except:
                likes = "0"

            # Extract Comments Count
            try:
                comments = driver.find_elements(By.XPATH, '//ul[@role="presentation"]/li')
                comment_count = len(comments)
            except:
                comment_count = 0

            # Add data to the list
            data.append({
                "Post ID": post_id,
                "Date": date,
                "Followers": followers,
                "Likes": likes,
                "Comments": comment_count,
                "Post Link": post_url
            })

        except Exception as e:
            print(f"Error processing {post_url}: {e}")

    return data

# Extract data
profile_url = "https://www.instagram.com/aadesh_charitable_foundation/"
num_posts = 10
posts_data = extract_instagram_data(profile_url, num_posts=num_posts)

# Convert to DataFrame
df = pd.DataFrame(posts_data)

# Save to Excel
excel_file = "instagram_data.xlsx"
df.to_excel(excel_file, index=False)
print(f"Data saved to {excel_file}")

# Quit the driver
driver.quit()


Data saved to instagram_data.xlsx
